### Gradiente estocástico descendiente
#### Equipo No. 6: Elena Villalobos Nolasco, Carolina Acosta Tovany y Aide Jazmín González Cruz

#### 1.- Se importan los paquetes

In [2]:
import random
import matplotlib.pyplot as plt
import csv
from math import exp
import numpy as np
import pandas as pd

#### 2.- Cargamos los datos

In [3]:
def load_data(path, header):
    marks_df = pd.read_csv(path, header=header)
    return marks_df

data = load_data("iris_num.txt",None)
data = np.array(data)

#### 3.- Definimos nuestra función costo/objetivo (LOSS Function)

$J(\theta) = \frac{1}{n} \sum_{i=1}^{n} L_h (y^{(i)}\theta * x^{(i)}) + \frac{\lambda}{2} || \theta || ^2$

$ L_h (v) = \begin{bmatrix}
1 -v \:\:\:\:\:  v < 1\\
0 \:\:\:\:\: otro\:caso
\end{bmatrix}$

In [4]:
def costo(data, n, theta, lamb):
    
    
    
    def Lh(theta,x,y):  # De acuerdo a la clase:
        v = np.dot(theta,x.T)*y
        if (v)<1:
            return 1 - v
        else:
            return 0
    
    costo = 0
    
    for i in data:
        x = i[:len(i)-1]
        y = i[4]
        
        costo += Lh(theta,x,y)+((lamb/2)*np.linalg.norm(theta)**2)
    
    costo *= 1/n
    
    return costo

#### 4.- Definimos la función SGD.

Para $i \in {1,...,n}$ aleatoria:

$\theta \leftarrow \theta-\eta_t \Delta_{\theta} \left[ Loss_h (y^{(i)} \theta  \dot x^{(i)} + \frac{\lambda}{2} ||\theta|| \right] $


Y de acuerdo a la clase:


$ L_h^{'} (v) = \begin{bmatrix}
-1 \:\:\:\:\:  v < 1\\
0 \:\:\:\:\: otro\:caso
\end{bmatrix}$


$\theta^t \leftarrow \theta^{t+1}-\eta_t L_h^{'} (v) + \lambda \theta$

In [37]:
def sgd(train_data,theta,lamb,iteraciones):
    
    x = np.array(train_data[:,0:4])
    y = np.array(train_data[4])
    y_len = len(y) # 
    
    def f_perdida(theta,x,y):  # De acuerdo a la clase:
        if (np.dot(theta,x.T)*y)<1:
            return -1
        else:
            return 0
    
    for j in range(iteraciones):
        random.seed(1993)
        rand_int = np.random.randint(0,y_len) # elige el dato al azar entre cero y y_len
        x_i=x[rand_int,:].reshape(1,x.shape[1])
        y_i=y[rand_int].reshape(1,1)
        #theta=theta-(1/(j+1))*y_i@x_i*(f_perdida(theta,x_i,y_i)+(lamb)*theta)
        theta=theta-(1/(j+1))*y_i@x_i*(f_perdida(theta,x_i,y_i)+(lamb/2)*np.linalg.norm(theta)**2)
        
    return theta

#### 5.- Definimos la función CrossValidation

In [38]:
random.seed(1993)

def CrossValidation(data,K):
    lamda_list = [1,0.01,0.001]
    #eta = 0.01 # LearningRate
    iteraciones = 1000
    #df_lamda = pd.DataFrame(0.0, index=range(1),columns=("100","10","1","0.1"))
    df_lamda = pd.DataFrame(0.0, index=range(1),columns=("1","0.01","0.001"))
    
    # Dividimos los datos en chunk de igual tamaño
    chunks = np.array_split(data, K)
    for lamda_ in lamda_list:
        error = 0.0
        for i in range(1,int(K)):
            
            # Inicializamos las tetas
            theta = np.array([0.0, 0.0, 0.0, 0.0])
            
            # El chunk i será nuestro bloque de datos de prueba
            test_data = chunks[i]
            
            # Entrenamos con todos los otros datos menos con el chunk[i]
            train_data = chunks.copy()
            train_data.pop(i) # Elimina el chunk i
            train_data = np.concatenate(train_data)
        
            
            theta_est = sgd(train_data, theta, lamda_, iteraciones)
            
            
            #  Obtenemos el error con la función objetivo
            error += costo(test_data,len(test_data), theta_est[0],lamda_)
        error /= K
        
        df_lamda[str(lamda_)][0] = error
        
    print(df_lamda)
    return (df_lamda.idxmin(axis=1)[0])


#### 6.- Hacemos la validación cruzada sobre nuestra data

In [35]:
df_lamda = CrossValidation(data,5.0)
float(df_lamda)

/home/carotova/.pyenv/versions/3.7.4/envs/machine-learning/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in double_scalars
  


     1      0.01     0.001
0  inf  0.674257  0.686069


0.01

#### Conclusión:

De acuerdo al CrossValidation , debemos utilizar lambda 0.01 para obtener mejor resultados, porque tendrá menor error segun la funcion de costo. 

Estamos haciendo 1000 iteraciones.

#### 7.- Volvemos a entrenar

In [40]:
#Se revuelven los datos
random.seed(1992)
random.shuffle(data)

#Datos de entrenamiento
datos_entrenamiento = data[0:120]

#Datos de verificación
data_validacion = data[121:150]


theta = np.array([0.0, 0.0, 0.0, 0.0])
iteraciones = 1000
lam = 0.01

theta_est = sgd(data_validacion, theta, lam, iteraciones)
print("Teta : ", theta_est[0])

Teta :  [-0.08831484 -0.83909116  3.31222094  1.0967151 ]
